## FUNCTION
### Given Path and value name, we get the summary of all the stations

In [1]:
def read_ecad_sum(path, VAL):
    import pandas as pd
    import numpy as np
    ecad = pd.read_csv(path, names = ['STAID', 'SOUID', 'DATE', VAL, 'QUALITY'], skiprows = 21, parse_dates = ['DATE']).set_index('DATE')
    ecad = ecad[ecad['QUALITY'] == 0]
    output = pd.DataFrame()
    if ecad[VAL].count() >= 365 * 20 and ecad.index.max() >= pd.to_datetime('01/01/2000') and ecad.index.min() < pd.to_datetime('01/01/2000'):
        output['Monthly'] = ecad[VAL].resample(rule = 'M').sum()
        output['Month'] = output.index.month
        months = np.arange(12)
        for i in range(0, 12, 1):
            months[i] = output[output["Month"] == i + 1].mean().drop(['Month'])
        station = ecad['STAID'].mean()
        result = pd.DataFrame(data = months, index = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], columns = [station]).T
    else:
        result = np.zeros(12)
        station = np.zeros(1)
    return result, station

def read_ecad_mean(path, VAL):
    import pandas as pd
    import numpy as np
    ecad = pd.read_csv(path, names = ['STAID', 'SOUID', 'DATE', VAL, 'QUALITY'], skiprows = 21, parse_dates = ['DATE']).set_index('DATE')
    ecad = ecad[ecad[VAL] != -9999]
    output = pd.DataFrame()
    if ecad[VAL].count() >= 365 * 20 and ecad.index.max() >= pd.to_datetime('01/01/2000') and ecad.index.min() < pd.to_datetime('01/01/2000'):
        output['Monthly'] = ecad[VAL].resample(rule = 'M').mean()
        output['Month'] = output.index.month
        months = np.arange(12)
        for i in range(0, 12, 1):
            months[i] = output[output["Month"] == i + 1].mean().drop(['Month'])
        station = ecad['STAID'].mean()
        result = pd.DataFrame(data = months, index = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], columns = [station]).T
    else:
        result = np.zeros(12)
        station = np.zeros(1)
    return result, station

def PM(m, Tmean, Tmax, Tmin, RH, U, RS, phi, h):
    import numpy as np
    julian = {'jan': 16, 'feb' : 44, 'mar' : 75, 'apr' : 105, 'may' : 136, 'jun': 166, 'jul' : 197, 'aug' : 228, 'sep' : 258, 'oct' : 289, 'nov': 319, 'dec': 350}    
    U2 = U * 0.1
    Tmean = Tmean * 0.1
    Tmin = Tmin * 0.1
    Tmax = Tmax * 0.1
    es1 = 0.6108 * np.exp((17.27 * Tmax)/(Tmax + 237.3))
    es2 = 0.6108 * np.exp((17.27 * Tmin)/(Tmin + 237.3))
    es = (es1 + es2) / 2
    ea = RH * es/100
    Delta = 4099 * (0.6108 * np.exp((17.27 * Tmean) / (Tmean + 237.3)) / ((Tmean + 237.3)**2))
    gamma = 0.665 * 0.001 * 1013 / 10
    dr = 1 + 0.033 * np.cos(2 * np.pi * julian[m] / 365)
    d = 0.4093 * np.sin((2 * np.pi * julian[m] / 365) - 1.39)
    omega_s = np.arccos(-1 * np.tan(phi) * np.tan(d))
    Ra = (24 * 60 * 0.082 * dr / np.pi) * (omega_s * np.sin(phi) * np.sin(d) + np.cos(phi) * np.cos(d) * np.sin(omega_s))
    Rso = (0.75 + 2 * 0.00001 * h) * Ra
    Rs = 0.0864 * RS
    Rns = 0.77 * Rs
    Rnl = 4.903 * 0.000000001 * (((273.16 + Tmax)**4 + (273.16 + Tmin)**4) / 2) * (0.34 - 0.14 * np.sqrt(ea)) * (1.35 * (Rs/Rso) - 0.35)
    Rn = Rns - Rnl
    ETo = (0.408 * Delta * (Rn - 0)+(900 * gamma * U2 * (es - ea) / (273 + Tmean))) / (Delta + gamma * (1 + 0.34 * U2))
    return ETo

def TW(T, phi, C = 1):
    days = pd.DataFrame(data = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31], index = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], columns = ['n'])
    days['julian'] = [16, 44, 75, 105, 136, 166, 197, 228, 258, 289, 319, 350]
    Tmean = T.ravel() * 0.1
    j = np.zeros(len(Tmean))
    d = np.zeros(len(Tmean))
    omega_s = np.zeros(len(Tmean))
    N = np.zeros(len(Tmean))
    Ep = np.zeros(len(Tmean))

    for i in range(len(Tmean)):
        d[i] = 0.4093 * np.sin((2 * np.pi * days['julian'][i] / 365) - 1.39)
        omega_s[i] = np.arccos(-1 * np.tan(phi) * np.tan(d[i]))
        N[i] = 24 * omega_s[i] / np.pi

    for i in range(len(Tmean)):
        if Tmean[i] < 0 :
            j[i] = 0
        else:
            j[i] = (Tmean[i] / 5) ** 1.514

    J = j.sum()
    a = (6.75 * 10 ** -7) * J ** 3 - (7.71 * 10 ** -5) * J ** 2 + (1.79 * 10 ** -2) * J + 0.492

    for i in range(len(Tmean)):
        if Tmean[i] < 0 :
            Ep[i] = 0            
        else:
            Ep[i] = (16 * (10 * Tmean[i] / J) ** a) * (days['n'][i] * N[i] / (12 * 30))
            Ep[i] = Ep[i] / days['n'][i] * C

    thrn = pd.DataFrame(Ep, index = n, columns = ['Ep Thornthwaite'])
    return thrn

# Run of function

## Precipitation

In [2]:
# import os
# import pandas as pd
# import numpy as np

# entries = os.listdir(r'C:\Users\owner\Videos\ECA_blend_rr')

# path = os.path.join(r"C:\Users\owner\Videos", "ECA_blend_rr")
# path = path + "\\"

# precipitation = np.zeros((len(entries), 12))
# station = np.zeros((len(entries), 1))

# for i in range(0, len(entries)):
#     precipitation[i], station[i] = read_ecad_sum(path + entries[i], 'p')
    
# prec = pd.DataFrame(data = precipitation, columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], index = station.ravel())
# prec.index.rename('Station ID', inplace = True)
# prec['sum'] = prec.sum(axis = 1)
# prec = prec[(prec.T != 0).any()]
# prec

## Wind Speed

In [ ]:
import os
import pandas as pd
import numpy as np

entries = os.listdir(r"C:\Dimos\database\ECA_blend_fg")

path = os.path.join(r"C:\Dimos\database", "ECA_blend_fg")
path = path + "\\"

ws = np.zeros((len(entries), 12))
station_ws = np.zeros((len(entries), 1))

for i in range(0, len(entries)):
    ws[i], station_ws[i] = read_ecad_mean(path + entries[i], 'U')

wind = pd.DataFrame(data = ws, columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], index = station_ws.ravel())
wind.index.rename('Station ID', inplace = True)
wind['mean'] = wind.mean(axis = 1)
wind = wind[(wind.T != 0).any()]
wind

## Relative Humidity

In [ ]:
import os
import pandas as pd
import numpy as np

entries = os.listdir(r"C:\Dimos\database\ECA_blend_hu")

path = os.path.join(r"C:\Dimos\database", "ECA_blend_hu")
path = path + "\\"

RelH = np.zeros((len(entries), 12))
station_RH = np.zeros((len(entries), 1))

for i in range(0, len(entries)):
    RelH[i], station_RH[i] = read_ecad_mean(path + entries[i], 'RH')

RH = pd.DataFrame(data = RelH, columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], index = station_RH.ravel())
RH['mean'] = RH.mean(axis = 1)
RH = RH[(RH.T != 0).any()]
RH

## Radiation

In [ ]:
import os
import pandas as pd
import numpy as np

entries = os.listdir(r"C:\Dimos\database\ECA_blend_qq")

path = os.path.join(r"C:\Dimos\database", "ECA_blend_qq")
path = path + "\\"

RS_ = np.zeros((len(entries), 12))
station_RS = np.zeros((len(entries), 1))

for i in range(0, len(entries)):
    RS_[i], station_RS[i] = read_ecad_mean(path + entries[i], 'RS')

RS = pd.DataFrame(data = RS_, columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], index = station_RS.ravel())
RS.index.rename('Station ID', inplace = True)
RS['mean'] = RS.mean(axis = 1)
RS = RS[(RS.T != 0).any()]
RS

## Mean temperature

In [ ]:
import os
import pandas as pd
import numpy as np

entries = os.listdir(r"C:\Dimos\database\ECA_blend_tg")

path = os.path.join(r"C:\Dimos\database", "ECA_blend_tg")
path = path + "\\"

Tm = np.zeros((len(entries), 12))
station_T = np.zeros((len(entries), 1))

for i in range(0, len(entries)):
    Tm[i], station_T[i] = read_ecad_mean(path + entries[i], 'Tmean')
    
T = pd.DataFrame(data = Tm, columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], index = station_T.ravel())
T.index.rename('Station ID', inplace = True)
T['mean'] = T.mean(axis = 1)
T = T[(T.T != 0).any()]
T

## Minimum temperature

In [ ]:
import os
import pandas as pd
import numpy as np

entries = os.listdir(r"C:\Dimos\database\ECA_blend_tn")

path = os.path.join(r"C:\Dimos\database", "ECA_blend_tn")
path = path + "\\"

Tminimum = np.zeros((len(entries), 12))
station_Tmin = np.zeros((len(entries), 1))

for i in range(0, len(entries)):
    Tminimum[i], station_Tmin[i] = read_ecad_mean(path + entries[i], 'Tmin')
    
Tmin = pd.DataFrame(data = Tminimum, columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], index = station_Tmin.ravel())
Tmin.index.rename('Station ID', inplace = True)
Tmin['mean'] = Tmin.mean(axis = 1)
Tmin = Tmin[(Tmin.T != 0).any()]
Tmin

## Maximum temperature

In [ ]:
import os
import pandas as pd
import numpy as np

entries = os.listdir(r"C:\Dimos\database\ECA_blend_tx")

path = os.path.join(r"C:\Dimos\database", "ECA_blend_tx")
path = path + "\\"

Tmaximum = np.zeros((len(entries), 12))
station_Tmax = np.zeros((len(entries), 1))

for i in range(0, len(entries)):
    Tmaximum[i], station_Tmax[i] = read_ecad_mean(path + entries[i], 'Tmax')
    
Tmax = pd.DataFrame(data = Tmaximum, columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], index = station_Tmax.ravel())
Tmax.index.rename('Station ID', inplace = True)
Tmax['mean'] = Tmax.mean(axis = 1)
Tmax = Tmax[(Tmax.T != 0).any()]
Tmax

## Export

In [ ]:
T.to_csv(r'C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\validation ECA&D\T.txt', sep=';', index=True, header=True)
Tmax.to_csv(r'C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\validation ECA&D\Tmax.txt', sep=';', index=True, header=True)
Tmin.to_csv(r'C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\validation ECA&D\Tmin.txt', sep=';', index=True, header=True)
wind.to_csv(r'C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\validation ECA&D\U.txt', sep=';', index=True, header=True)
RH.to_csv(r'C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\validation ECA&D\RH.txt', sep=';', index=True, header=True)
RS.to_csv(r'C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\validation ECA&D\QQ.txt', sep=';', index=True, header=True)

# Find mutual stations

In [ ]:
hu = RH
qq = RS
tmeann = T
tmaxx = Tmax
tminn = Tmin
windspeed = wind

idx = qq.index.intersection(hu.index)
idx2  = windspeed.index.intersection(tmeann.index)
idx3 = tmaxx.index.intersection(tminn.index)
idx = pd.DataFrame(index = idx)
idx2  = pd.DataFrame(index = idx2)
idx3 = pd.DataFrame(index = idx3)
mutuals = idx.index.intersection(idx2.index)
mutuals = pd.DataFrame(index = mutuals)
mutuals = mutuals.index.intersection(idx3.index)
mutuals = pd.DataFrame(index = mutuals)
tmeann = pd.read_csv(r'C:\Dimos\database\whole data\Tmean.txt', skiprows = 18, names = ['STAID','Station Name', 'CN', 'Lat', 'Lon', 'h']).set_index('STAID')
stations_all = mutuals.merge(tmeann, left_index = True, right_index = True)
phi = np.zeros((len(stations_all['Lat']), 1))
for i in range(len(stations_all['Lat'])):
    phi[i] = (int(stations_all['Lat'].iloc[i][0:3]) + int(stations_all['Lat'].iloc[i][4:6])/60 + int(stations_all['Lat'].iloc[i][7:9])/3600)
stations_all['phi'] = phi * np.pi / 180
stations_all.to_csv(r'C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\validation ECA&D\stations.txt', sep=';', index=True, header=True)
stations_all


## Apply Penman Monteith

In [ ]:
evap_jan, evap_feb, evap_mar, evap_apr, evap_may, evap_jun, evap_jul, evap_aug, evap_sep, evap_oct, evap_nov, evap_dec = ([] for i in range(12))
for i in stations_all.index:
    evap_jan.append(PM('jan', T.loc[i:i, 'jan'], Tmax.loc[i:i, 'jan'], Tmin.loc[i:i, 'jan'], RH.loc[i:i, 'jan'], wind.loc[i:i, 'jan'], RS.loc[i:i, 'jan'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_feb.append(PM('feb', T.loc[i:i, 'feb'], Tmax.loc[i:i, 'feb'], Tmin.loc[i:i, 'feb'], RH.loc[i:i, 'feb'], wind.loc[i:i, 'feb'], RS.loc[i:i, 'feb'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_mar.append(PM('mar', T.loc[i:i, 'mar'], Tmax.loc[i:i, 'mar'], Tmin.loc[i:i, 'mar'], RH.loc[i:i, 'mar'], wind.loc[i:i, 'mar'], RS.loc[i:i, 'mar'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_apr.append(PM('apr', T.loc[i:i, 'apr'], Tmax.loc[i:i, 'apr'], Tmin.loc[i:i, 'apr'], RH.loc[i:i, 'apr'], wind.loc[i:i, 'apr'], RS.loc[i:i, 'apr'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_may.append(PM('may', T.loc[i:i, 'may'], Tmax.loc[i:i, 'may'], Tmin.loc[i:i, 'may'], RH.loc[i:i, 'may'], wind.loc[i:i, 'may'], RS.loc[i:i, 'may'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_jun.append(PM('jun', T.loc[i:i, 'jun'], Tmax.loc[i:i, 'jun'], Tmin.loc[i:i, 'jun'], RH.loc[i:i, 'jun'], wind.loc[i:i, 'jun'], RS.loc[i:i, 'jun'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_jul.append(PM('jul', T.loc[i:i, 'jul'], Tmax.loc[i:i, 'jul'], Tmin.loc[i:i, 'jul'], RH.loc[i:i, 'jul'], wind.loc[i:i, 'jul'], RS.loc[i:i, 'jul'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_aug.append(PM('aug', T.loc[i:i, 'aug'], Tmax.loc[i:i, 'aug'], Tmin.loc[i:i, 'aug'], RH.loc[i:i, 'aug'], wind.loc[i:i, 'aug'], RS.loc[i:i, 'aug'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_sep.append(PM('sep', T.loc[i:i, 'sep'], Tmax.loc[i:i, 'sep'], Tmin.loc[i:i, 'sep'], RH.loc[i:i, 'sep'], wind.loc[i:i, 'sep'], RS.loc[i:i, 'sep'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_oct.append(PM('oct', T.loc[i:i, 'oct'], Tmax.loc[i:i, 'oct'], Tmin.loc[i:i, 'oct'], RH.loc[i:i, 'oct'], wind.loc[i:i, 'oct'], RS.loc[i:i, 'oct'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_nov.append(PM('nov', T.loc[i:i, 'nov'], Tmax.loc[i:i, 'nov'], Tmin.loc[i:i, 'nov'], RH.loc[i:i, 'nov'], wind.loc[i:i, 'nov'], RS.loc[i:i, 'nov'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)
    evap_dec.append(PM('dec', T.loc[i:i, 'dec'], Tmax.loc[i:i, 'dec'], Tmin.loc[i:i, 'dec'], RH.loc[i:i, 'dec'], wind.loc[i:i, 'dec'], RS.loc[i:i, 'dec'], stations_all.loc[i:i, 'phi'], stations_all.loc[i:i, 'h']).values)

In [ ]:
evap = pd.DataFrame(evap_jan, columns = ['Jan'], index = [stations_all.index])
evap['Feb'] = np.array(evap_feb).ravel()
evap['Mar'] = np.array(evap_mar).ravel()
evap['Apr'] = np.array(evap_apr).ravel()
evap['May'] = np.array(evap_may).ravel()
evap['Jun'] = np.array(evap_jun).ravel()
evap['Jul'] = np.array(evap_jul).ravel()
evap['Aug'] = np.array(evap_aug).ravel()
evap['Sep'] = np.array(evap_sep).ravel()
evap['Oct'] = np.array(evap_oct).ravel()
evap['Nov'] = np.array(evap_nov).ravel()
evap['Dec'] = np.array(evap_dec).ravel()
evap = evap[(evap >= 0).all(1)]
evap.to_csv(r'C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\validation ECA&D\PM_ETo.txt', sep=';', index=True, header=True)
evap

## APPLY THORNTHWAITE

In [ ]:
T2 = T.drop(['mean'], axis = 1)
THORN = []
for i in stations_all.index:
    THORN.append((TW(T2.loc[i:i].values, stations_all.loc[i:i, 'phi']).values).T)
evap_th = pd.DataFrame((np.array(THORN).ravel()).reshape(len(stations_all.index), 12), columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], index = stations_all.index)
evap_th = evap_th[(evap_th >= 0).all(1)]
evap_th.to_csv(r'C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\validation ECA&D\TW_ETo.txt', sep=';', index=True, header=True)
evap_th

## Plot station network

In [ ]:
stations = pd.read_csv(r"C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\validation ECA&D\stations.txt", delimiter = ';', index_col = 0)
phi = np.zeros(len(stations['Lat']))
labda = np.zeros(len(stations['Lat']))

for i in range(len(stations)):
    phi[i] = (int(stations['Lat'].iloc[i][0:3]) + int(stations['Lat'].iloc[i][4:6])/60 + int(stations['Lat'].iloc[i][7:9])/3600)
    labda[i] = (int(stations['Lon'].iloc[i][0:4]) + int(stations['Lon'].iloc[i][5:7])/60 + int(stations['Lon'].iloc[i][8:10])/3600)

import os
os.environ['PROJ_LIB'] = 'C:/Users/owner/Anaconda3/Lib/site-packages/mpl_toolkits/basemap'

from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize = (20, 20))
axs = plt.subplot(111)
m=Basemap()
 
# Then add element: draw coast line, map boundary, and fill continents:
m.drawcoastlines()
m.drawmapboundary()
m.fillcontinents()
parallels = np.arange(-90.,91.,15.)
m.drawparallels(parallels,labels=[False,True,True,False])
meridians = np.arange(-180.,181.,25.)
m.drawmeridians(meridians,labels=[True,False,False,True])
m.drawmapboundary(fill_color='white')
axs.plot(labda, phi, 'o')
plt.plot();

In [ ]:
evap.describe()

In [ ]:
evap_th.describe()